## Deep Neural Network in PyTorch - Tabular Data

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.utils.data as data_utils 
from torch.utils.tensorboard import SummaryWriter


import itertools

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=100
pd.options.display.max_columns=100

In [34]:
data = pd.read_csv("creditcard.csv")

In [35]:
data.isnull().values.any()

False

In [36]:
data['Class'].value_counts() / len(data) *100

0    99.827251
1     0.172749
Name: Class, dtype: float64

In [37]:
data.drop(['Time'],inplace=True,axis=1)

In [38]:
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))
## purpose -1 is when you don't know the shape !

In [39]:
x_data = data.loc[:,data.columns!='Class']
y_data = data.loc[:,data.columns=='Class']

In [54]:
x_train,x_test,y_train, y_test = train_test_split(x_data,y_data,test_size=.2, stratify=y_data)

In [55]:
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [56]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((227845, 29), (56962, 29), (227845, 1), (56962, 1))

In [57]:
x_train_dataset = torch.tensor(x_train.values.astype(np.float32))
x_test_dataset = torch.tensor(x_test.values.astype(np.float32))


In [58]:
x_train_dataset.shape, x_test_dataset.shape

(torch.Size([227845, 29]), torch.Size([56962, 29]))

In [66]:
x_train_dataset[0:1]

tensor([[-0.7354,  1.3289,  0.9788,  2.6665,  1.0574,  0.7564,  0.6604,  0.3192,
         -1.2633,  1.2117, -0.1439, -0.6406, -1.8544,  0.2604, -2.2951,  0.6518,
         -0.7094,  0.2060, -1.2517, -0.2179,  0.1190,  0.3273, -0.3052,  0.6546,
          0.3442,  0.0659, -0.0323,  0.0650, -0.3219]])

In [63]:
y_train_dataset = torch.tensor(y_train.values.astype(np.float32))
y_test_dataset = torch.tensor(y_test.values.astype(np.float32))

In [64]:
y_train_dataset.shape, y_test_dataset.shape

(torch.Size([227845, 1]), torch.Size([56962, 1]))

In [67]:
y_train_dataset[0:1]

tensor([[0.]])

In [ ]:
## Datasets and Dataloaders in PyTorch:
"""
Dataset stores the samples and their corresponding labels.
DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

These 2 libraries also allow you to download and prepare pre-loaded datasets from PyTorch

The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in “minibatches”, 
reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval.
DataLoader is an iterable that abstracts this complexity for us in an easy API.

In [71]:
train_dataset = data_utils.TensorDataset(x_train_dataset, y_train_dataset)
test_dataset = data_utils.TensorDataset(x_test_dataset, y_test_dataset)

In [72]:
train_dataset

In [177]:
train_dataloader = data_utils.DataLoader(train_dataset, batch_size = 256 )
test_dataloader = data_utils.DataLoader(test_dataset, batch_size = 256)

In [178]:
train_dataloader

In [179]:
example = iter(train_dataloader)

In [180]:
samples, labels = example.next()

In [181]:
samples.shape ## 32 is the bathc_size

torch.Size([256, 29])

In [182]:
labels.shape

torch.Size([256, 1])

In [183]:
labels[0:5], y_train[0:5]

(tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.]]),
    Class
 0      0
 1      0
 2      0
 3      0
 4      0)

In [184]:
samples[0], x_train_dataset[0] 

## NOTE Shuffle isn't True while prepareing the dataloader. If shuffle is true, then this wouldn't match

(tensor([-0.7354,  1.3289,  0.9788,  2.6665,  1.0574,  0.7564,  0.6604,  0.3192,
         -1.2633,  1.2117, -0.1439, -0.6406, -1.8544,  0.2604, -2.2951,  0.6518,
         -0.7094,  0.2060, -1.2517, -0.2179,  0.1190,  0.3273, -0.3052,  0.6546,
          0.3442,  0.0659, -0.0323,  0.0650, -0.3219]),
 tensor([-0.7354,  1.3289,  0.9788,  2.6665,  1.0574,  0.7564,  0.6604,  0.3192,
         -1.2633,  1.2117, -0.1439, -0.6406, -1.8544,  0.2604, -2.2951,  0.6518,
         -0.7094,  0.2060, -1.2517, -0.2179,  0.1190,  0.3273, -0.3052,  0.6546,
          0.3442,  0.0659, -0.0323,  0.0650, -0.3219]))

In [185]:
input_size = x_train.shape[1]
hidden_size = 100
num_epochs = 2
batch_size = 2891*256
learning_rate = .001

In [259]:
class DeepNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(DeepNeuralNet,self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu1= nn.ReLU()
        self.l2 = nn.Linear(hidden_size, int(hidden_size/2))
        self.relu2= nn.ReLU()
        hidden_size = int(hidden_size/2)
        self.l3 = nn.Linear(hidden_size,1)
        ## In mulitclas with pyTorch, we don't define the softmax layer since it is taken care by the loss function, which is cross-entropy
 
    def forward(self, x):
        out = self.l1(x)
        out = self.relu1(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.l3(out)
        y_pred = torch.sigmoid(out)
        return y_pred

In [260]:
mymodel = DeepNeuralNet(input_size,hidden_size)

In [261]:
mymodel

DeepNeuralNet(
  (l1): Linear(in_features=29, out_features=100, bias=True)
  (relu1): ReLU()
  (l2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (l3): Linear(in_features=50, out_features=1, bias=True)
)

In [262]:
mymodel.parameters

<bound method Module.parameters of DeepNeuralNet(
  (l1): Linear(in_features=29, out_features=100, bias=True)
  (relu1): ReLU()
  (l2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (l3): Linear(in_features=50, out_features=1, bias=True)
)>

In [263]:
criterion = nn.BCELoss() ## Binary Cross Entropy Loss
optimizer = torch.optim.Adam(mymodel.parameters(),lr = learning_rate)

In [289]:
x_train_dataset.shape[0]/256 ## meaning 890 batches

890.01953125

In [264]:
n_total_steps = len(train_dataloader)
n_total_steps

891

In [280]:
writer

In [295]:
count=0
writer = SummaryWriter()
for epoch in range(num_epochs):
    for i,(x_data, labels) in enumerate(train_dataloader):
        count+=1
        y_predicted = mymodel(x_data) ## forward propagation
        
        loss = criterion(y_predicted,labels) ## loss calculation
        writer.add_scalar("Loss/train", loss, epoch)
        
        optimizer.zero_grad() ## Empty the gradients
        
        loss.backward() ## backward propagation
    
        optimizer.step() ## update the weights
        
        if (i+1)%100 == 0:
                print(f'{epoch+1} of {num_epochs} ; step: {i+1} of {n_total_steps} ; loss = {loss.item():.4f}')
    print(count)
writer.flush()
writer.close()

1 of 2 ; step: 100 of 891 ; loss = 0.0080
1 of 2 ; step: 200 of 891 ; loss = 0.0000
1 of 2 ; step: 300 of 891 ; loss = 0.0000
1 of 2 ; step: 400 of 891 ; loss = 0.0001
1 of 2 ; step: 500 of 891 ; loss = 0.0003
1 of 2 ; step: 600 of 891 ; loss = 0.0000
1 of 2 ; step: 700 of 891 ; loss = 0.0001
1 of 2 ; step: 800 of 891 ; loss = 0.0001
891
2 of 2 ; step: 100 of 891 ; loss = 0.0081
2 of 2 ; step: 200 of 891 ; loss = 0.0000
2 of 2 ; step: 300 of 891 ; loss = 0.0000
2 of 2 ; step: 400 of 891 ; loss = 0.0001
2 of 2 ; step: 500 of 891 ; loss = 0.0003
2 of 2 ; step: 600 of 891 ; loss = 0.0000
2 of 2 ; step: 700 of 891 ; loss = 0.0001
2 of 2 ; step: 800 of 891 ; loss = 0.0001
1782


In [300]:
y_predicted.shape

torch.Size([5, 1])

In [303]:
type(y_predicted[0].item())

float

In [296]:
count

1782

In [266]:
x_test_dataset[637]

tensor([ 1.7079,  0.0249, -0.4881,  3.7875,  1.1395,  2.9147, -0.7434,  0.6991,
         1.0085,  0.9128,  0.7654, -2.0439,  1.0015,  1.3229, -2.7201, -0.1534,
         0.7526, -0.7558, -1.9126, -0.3680,  0.0109,  0.5483,  0.0912, -1.0080,
        -0.0822,  0.1797,  0.0077, -0.0688, -0.2183])

In [267]:
## Test Prediction
pred_list=[]
with torch.no_grad():
    for samples,labels in test_dataloader:
        outputs = mymodel(samples)
        
        predictions,_ = torch.max(outputs,1)
        predict_batch_list = list(predictions.detach().numpy())
        pred_list.append(predict_batch_list)
        
pred_list_final = list(itertools.chain(*pred_list))           

In [268]:
len(pred_list_final)

56962

In [269]:
x_test.shape

(56962, 29)

In [270]:
from sklearn.metrics import roc_auc_score

In [271]:
roc_auc_score(y_test_dataset,pred_list_final)

0.973344743778209

In [273]:
## TensorBoard

In [ ]:
"""
Note: Having TensorFlow installed is not a prerequisite to running TensorBoard, although it is a product of the TensorFlow ecosystem, 
TensorBoard by itself can be used with PyTorch.


In [274]:
from torch.utils.tensorboard import SummaryWriter

In [275]:
!tensorboard --logdir runs

^C


In [ ]:
## Backup & Experiment

In [ ]:
y_train_dataset[820]

tensor([1.])

In [221]:
mymodel(x_train_dataset[820])

tensor([0.9731], grad_fn=<SigmoidBackward0>)

In [231]:
mymodel(x_train_dataset[820:823])

tensor([[9.7308e-01],
        [1.7608e-05],
        [5.2655e-05]], grad_fn=<SigmoidBackward0>)

In [ ]:
torch.max(mymodel(x_train_dataset[820:823]),1)

torch.return_types.max(
values=tensor([9.7308e-01, 1.7608e-05, 5.2655e-05], grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0]))

In [ ]:
_,x=torch.max(mymodel(x_train_dataset[820:823]),1)

In [ ]:
list(_.detach().numpy())

[0.97308254, 1.7608267e-05, 5.2654636e-05]